# Quadratic Optimization
Objective Function is Quadratic and Constraints are linear

In [1]:
import docplex.mp
import random

# first import the Model class from docplex.mp
from docplex.mp.model import Model

In [77]:
# create one model instance, with a name
m = Model(name='AMC Quadratic Optimization')

# by default, all variables in Docplex have a lower bound of 0 and infinite upper bound
x1 = m.binary_var(name='x1')
x2 = m.binary_var(name='x2')
x3 = m.binary_var(name='x3')
# write constraints
m.add_constraint(x1 + 2 * x2 - x3 <= 3)
m.add_constraint(x1 + x2 + x3 >= 2)
m.add_constraint(x2 - x3 <= 1)

m.maximize(2 * x1*x3 + 3 * x2 - 6*x3)

In [2]:
def getLinearExpression(model, variableList, coeffList):
    from docplex.mp.linear import LinearExpr
    lnExpr = LinearExpr(model)
    if len(coeffList)>0:
        count=0
        for var in variableList:
            lnExpr.add(coeffList[count]*var)
            count=count+1
    else:
        for var in variableList:
            lnExpr.add(var)
        
    return (lnExpr)

def get_generic_MIQP(num_binaryVars, num_continuousVars):
    # construct model using docplex
    m = Model('AMC Quadratic Optimization')

    if num_binaryVars < 1 or num_continuousVars < 1:
        print('Both the Binary Variables and Continuous Variables should be at least 1')
        print('Using 1 binary variable and 1 continuous variable')
        num_binaryVars=1
        num_continuousVars=1
        
    constraints = 1
    binary_size = num_binaryVars
    size = num_continuousVars
    if (int)((num_binaryVars+num_continuousVars)/3) > 1:
        constraints = random.randint(1, (int)((2*num_binaryVars+num_continuousVars)/2.5))
    x_vals = [random.randint(0,1) for i in range(binary_size)]
    y_vals = [random.randint(0,1) for i in range(binary_size)]
    u_vals = [random.randint(0,20) for i in range(size)]
    xs = m.binary_var_list(binary_size, name='x')
    ys = m.binary_var_list(binary_size, name='y')
    us = m.continuous_var_list(size, name='u')

    # Binary and Continuous variable must be separable in the objective
    objQ=0
    for x in xs:
        for y in ys:
            makeQuadratic=random.choice([True,False,False,False,False,True,False, False])
            if makeQuadratic:
                objQ=objQ+ x*y
    obj = objQ + m.sum([xs[i] for i in range(binary_size)]) + m.sum([ys[i] for i in range(binary_size)]) + m.sum([us[i] for i in range(size)])
    m.minimize(obj)
    for i in range(constraints):
        xList = random.choices(range(binary_size), k = 3)  
        yList = random.choices(range(binary_size), k = 2)  
        uList = random.choices(range(size), k = 2)
        variableList=[]
        coeffList=[]
        value=0
        for i in xList:
            variableList.append(xs[i])
            coeff=random.randint(1,90)
            coeffList.append(coeff)
            value = value + coeff*x_vals[i]
        for i in yList:
            variableList.append(ys[i])
            coeff=random.randint(1,90)
            coeffList.append(coeff)
            value = value + coeff*y_vals[i]
        for i in uList:
            variableList.append(us[i])
            coeff=random.randint(1,90)
            coeffList.append(coeff)
            value = value + coeff*u_vals[i]

        m.add_constraint(getLinearExpression(m,variableList,coeffList) == value, 'Constraint')
    return(m)


In [17]:
binary_vars=6
int_vars=0
continuous_vars=2
m = get_generic_MIQP(binary_vars,continuous_vars)

In [18]:
m.print_information()

Model: AMC Quadratic Optimization
 - number of variables: 14
   - binary=12, integer=0, continuous=2
 - number of constraints: 3
   - linear=3
 - parameters: defaults
 - objective: minimize quadratic
 - problem type is: MIQP


In [19]:
print(m.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Quadratic Optimization

Minimize
 obj: x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + y_0 + y_1 + y_2 + y_3 + y_4 + y_5
      + u_0 + u_1 + [ 2 x_0*y_2 + 2 x_0*y_4 + 2 x_1*y_1 + 2 x_1*y_3 + 2 x_3*y_0
      + 2 x_3*y_3 + 2 x_4*y_5 + 2 x_5*y_0 + 2 x_5*y_3 ]/2
Subject To
 Constraint: 39 x_0 + 39 x_5 + 14 y_2 + 89 y_5 + 100 u_0 = 581
 Constraint#1: 81 x_2 + 80 x_3 + 11 x_5 + 48 y_0 + 85 y_3 + 47 u_0 + 89 u_1 = 
               1561
 Constraint#2: 166 x_1 + 25 x_2 + 115 y_2 + 77 u_0 + 90 u_1 = 1528

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= y_0 <= 1
 0 <= y_1 <= 1
 0 <= y_2 <= 1
 0 <= y_3 <= 1
 0 <= y_4 <= 1
 0 <= y_5 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5 y_0 y_1 y_2 y_3 y_4 y_5
End



In [20]:
from docplex.mp.constants import int_probtype_to_string
filename='..\\problemFiles\\201-Quadratic Optimization Problem-'
varText=''
if m.number_of_binary_variables>0:
    varText='B%03d'%(m.number_of_binary_variables)+'-'
if m.number_of_integer_variables>0:    
    varText=varText+'I%03d'%(m.number_of_integer_variables)+'-'
if m.number_of_continuous_variables>0:    
    varText=varText+'C%03d'%(m.number_of_continuous_variables)+'-'
cpx = m.get_cplex(do_raise=False)
if cpx:
    cpx_probtype = cpx.get_problem_type()
    filename=filename+int_probtype_to_string(cpx_probtype)+'-'

filename=filename+varText+str(random.randint(122100,999999))
m.export_as_lp(filename)

'..\\problemFiles\\201-Quadratic Optimization Problem-MIQP-B012-C002-927308.lp'